In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.chdir(globals()['_dh'][0])
os.chdir('..')
print(os.path.abspath(os.curdir))

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import random 
import tensorflow as tf
from tqdm import tqdm
from scipy import signal as sg

import rfcutils # this corresponds to utility functions provided for the challenge

import rfcutils.ofdm_helper_fn_short as ofdmfn

from src import cnn_model_delay as cnn_delay

get_sinr = lambda s, i: 10*np.log10(np.mean(np.abs(s)**2)/np.mean(np.abs(i)**2))
get_pow = lambda s: np.mean(np.abs(s)**2)

---

In [ ]:
def eval_qpsk_demod(sig_est, sig_true, start_idx):
    qpsk_sig_idx = 16 - start_idx%16
    bit_est = rfcutils.matched_filter_demod(sig_est[qpsk_sig_idx:])
    bit_true = rfcutils.matched_filter_demod(sig_true[qpsk_sig_idx:])
    ber = np.sum(bit_est != bit_true)/len(bit_true)
    assert len(bit_est) == len(bit_true)
    return ber

def eval_sigs(sig_est, sig_true, start_idx):
    mse = get_pow(sig_est - sig_true)
    mse_db = 10*np.log10(mse)
    sdr = get_sinr(sig_true, sig_est-sig_true)
    ber = eval_qpsk_demod(sig_est, sig_true, start_idx)
    return (mse, mse_db, sdr, ber)

## Load models

In [ ]:
long_k_sz = 100
window_len_sync = 640
model_name = f'sync_mixture_OFDM64_CP16_QPSK_sig_len{window_len_sync}_K{long_k_sz}_V2batch'
print(f'Training {model_name}')
# input_shape = all_sig_mixture_train_comp.shape[1:]
synch_model = cnn_delay.get_cnn_model_delay_new((window_len_sync, 2), vec_delays=80, k_sz=21, long_k_sz=long_k_sz)
synch_model.load_weights(f'trained_models/sync_OFDM_model/{model_name}')

## CCN-Sync MMSE

In [ ]:
random.seed(10)
np.random.seed(10)

sig_len = 400
core_vec_len = sig_len-80

qpsk_dataset, sync_qpsk_dataset, sync2_qpsk_dataset, all_start_idx1 = [], [], [], []
for idx in tqdm(range(10000)):
    sig1, _, _, _ = rfcutils.generate_qpsk_signal(sig_len//16)
    sig1_core = sig1[:core_vec_len]
    start_idx1 = 0
    
    qpsk_dataset.append(sig1) 
    sync_qpsk_dataset.append(sig1_core)
    all_start_idx1.append(start_idx1)


ofdm_dataset, sync_ofdm_dataset, sync2_ofdm_dataset, all_start_idx2 = [], [], [], []
for idx in tqdm(range(10000)):
    data, _, _ = ofdmfn.generate_ofdm_signal(56*(sig_len//80*2))
    start_idx2 = np.random.randint(len(data)-sig_len)
    sig2 = data[start_idx2:start_idx2+sig_len]
    
#     sig2_segT_sync = data[start_idx2:start_idx2+window_len_sync]
#     sig2_bands_compT_sync = np.dstack((sig2_segT_sync.real, sig2_segT_sync.imag))
#     s_idx = np.argmax(synch_model(sig2_bands_compT_sync[:,:window_len_sync,:]))

    s_idx = 80 - start_idx2%80
    sig2_core = sig2[s_idx:s_idx+core_vec_len]
    sig2_core_ext = data[0:sig_len]
    ofdm_dataset.append(sig2) 
    sync_ofdm_dataset.append(sig2_core)
    sync2_ofdm_dataset.append(sig2_core_ext)
    all_start_idx2.append(start_idx2)
 
qpsk_dataset = np.array(qpsk_dataset)
ofdm_dataset = np.array(ofdm_dataset)

sync_qpsk_dataset = np.array(sync_qpsk_dataset)
sync_ofdm_dataset = np.array(sync_ofdm_dataset)
sync2_ofdm_dataset = np.array(sync2_ofdm_dataset)

In [ ]:
cov_qpsk = 1/np.shape(qpsk_dataset)[0]*np.matmul(qpsk_dataset.T, qpsk_dataset.conj())
cov_qpsk_sync = 1/np.shape(sync_qpsk_dataset)[0]*np.matmul(sync_qpsk_dataset.T, sync_qpsk_dataset.conj())
plt.figure(figsize=(8,8))
plt.subplot(121)
plt.imshow(np.abs(cov_qpsk))
plt.subplot(122)
plt.imshow(np.abs(cov_qpsk_sync))
# plt.savefig(os.path.join("figures", f"QPSK_cov1.eps"), dpi=300)

In [ ]:
cov_ofdm = 1/np.shape(ofdm_dataset)[0]*np.matmul(ofdm_dataset.T, ofdm_dataset.conj())
cov_ofdm_sync = 1/np.shape(sync_ofdm_dataset)[0]*np.matmul(sync_ofdm_dataset.T, sync_ofdm_dataset.conj())
cov_ofdm_sync_ext = 1/np.shape(sync2_ofdm_dataset)[0]*np.matmul(sync2_ofdm_dataset.T, sync2_ofdm_dataset.conj())

plt.figure(figsize=(12,18))
plt.subplot(131)
plt.imshow(np.abs(cov_ofdm))
plt.subplot(132)
plt.imshow(np.abs(cov_ofdm_sync))
plt.subplot(133)
plt.imshow(np.abs(cov_ofdm_sync_ext))
# plt.savefig(os.path.join("figures", f"OFDM_cov1.eps"), dpi=300)


In [ ]:
random.seed(42)
np.random.seed(42)
n_per_sinr = 1000
all_sinr = np.arange(-24,1.5,1.5)

all_sig_mixture, all_sig1, all_sig2, all_tau1, all_tau2 = [], [], [], [], []
for target_sinr in tqdm(all_sinr):
    for idx in range(n_per_sinr):
        data, _, _, _ = rfcutils.generate_qpsk_signal(sig_len//16)
        tau_s = 0
        sig1 = data[0:core_vec_len]
        
        data, _, _ = ofdmfn.generate_ofdm_signal(56*(window_len_sync//80*2))
        start_idx2 = np.random.randint(len(data)-window_len_sync)
        sig2 = data[start_idx2:start_idx2+core_vec_len]
        
        sig2_segT_sync = data[start_idx2:start_idx2+window_len_sync]
        sig2_bands_compT_sync = np.dstack((sig2_segT_sync.real, sig2_segT_sync.imag))
        tau_b = np.argmax(synch_model(sig2_bands_compT_sync[:,:window_len_sync,:]))
        
        coeff = np.sqrt(1/(10**(target_sinr/10)))

        sig_mixture = sig1 + sig2 * coeff
        all_sig_mixture.append(sig_mixture)
        all_sig1.append(sig1)
        all_sig2.append(sig2*coeff)
        
        all_tau1.append(tau_s)
        all_tau2.append(tau_b)
        
all_sig_mixture = np.array(all_sig_mixture)
all_sig1 = np.array(all_sig1)
all_sig2 = np.array(all_sig2)

In [ ]:
all_metric2 = []
nn_ber_mmse = []

for iidx in tqdm(range(all_sig_mixture.shape[0])):
    # initial estimate
    sinr = 10**(all_sinr[iidx//n_per_sinr]/10)
    
    tau_s = all_tau1[iidx] % 16
    tau_b = all_tau2[iidx] % 80
    Css = cov_qpsk_sync.copy()
    Cbb = cov_ofdm_sync_ext[80-tau_b:80-tau_b+core_vec_len, 80-tau_b:80-tau_b+core_vec_len]

    scaled_Cbb = Cbb * 1/sinr
    Cyy = Css + scaled_Cbb
    Csy = Css.copy()

    U,S,Vh = np.linalg.svd(Cyy,hermitian=True)
    sthr_idx = np.linalg.matrix_rank(Cyy) + 1
    Cyy_inv = np.matmul(U[:,:sthr_idx], np.matmul(np.diag(1.0/(S[:sthr_idx])), U[:,:sthr_idx].conj().T))
    W = np.matmul(Csy,Cyy_inv)
    s_est = np.matmul(W, all_sig_mixture[iidx])
    nn_ber_mmse.append((eval_sigs(s_est, all_sig1[iidx], 0))[-1])


In [ ]:
nsim_mmse = 1000

plt.figure()
SIR_values = np.arange(-24, 1.5, 1.5)
plt.semilogy(SIR_values,np.array(nn_ber_mmse).reshape(-1,nsim_mmse).mean(axis=-1), 'x-', label='L-MMSE sync')

plt.legend()
plt.title("BER OFDM64 MMSE sync")
plt.xlim(xmax = 0, xmin = -24)
plt.ylim(ymax = 1, ymin = 1e-4)
plt.grid()
# plt.savefig("ofdm64_40960_K101_TS1000_syncOFDM_vs_syncQPSK.pdf")

## Save data in csv files

In [ ]:
# Save BER
ber_mmse_data = np.stack((SIR_values,np.array(nn_ber_mmse).reshape(-1,nsim_mmse).mean(axis=-1)),axis=1)
np.savetxt("data/ber_mmse_data.csv", ber_mmse_data, delimiter=",")
